In [ ]:
#!pip install pymoo --q --user

# **Importation des bibliothéques**

In [15]:
from pymoo.problems import get_problem
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter

import matplotlib.pyplot as plt
import numpy as np

# **Initialisation d'algorithme**

In [4]:
Algorithm = PSO(
    pop_size=50,  # Nombre des particules
    w=0.5,        # Poids d'inertie
    c1=1.5,       # Coefficient cognitif
    c2=2.0        # Coefficient social
)

In [5]:
class MultiObjectivePSO:
    def __init__(self, problem, num_particles=50, max_gen=100, w=0.5, c1=1.5, c2=1.5):
        self.problem = problem
        self.num_particles = num_particles
        self.max_gen = max_gen
        self.w = w
        self.c1 = c1
        self.c2 = c2

        self.n_var = problem.n_var
        self.n_obj = problem.n_obj
        self.lower_bound = problem.xl
        self.upper_bound = problem.xu

        self.init_particles()

    def init_particles(self):
        self.positions = np.random.uniform(
            self.lower_bound, self.upper_bound,
            (self.num_particles, self.n_var)
        )
        self.velocities = np.random.uniform(
            -1, 1, (self.num_particles, self.n_var)
        )
        self.personal_best_positions = self.positions.copy()
        self.personal_best_scores = np.inf * np.ones((self.num_particles, self.n_obj))
        self.global_best_archive = []

    def evaluate(self, positions):
        F = np.zeros((positions.shape[0], self.n_obj))
        for i, x in enumerate(positions):
            F[i, :] = self.problem.evaluate(np.array([x]))[0]
        return F

    def dominates(self, a, b):
        return np.all(a <= b) and np.any(a < b)

    def update_archive(self, F, positions):
        for i in range(F.shape[0]):
            new_sol = (positions[i], F[i])
            self.global_best_archive = [
                sol for sol in self.global_best_archive if not self.dominates(new_sol[1], sol[1])
            ]
            if not any(self.dominates(sol[1], new_sol[1]) for sol in self.global_best_archive):
                self.global_best_archive.append(new_sol)

    def optimize(self):
        for gen in range(self.max_gen):
            scores = self.evaluate(self.positions)

            for i in range(self.num_particles):
                if self.dominates(scores[i], self.personal_best_scores[i]):
                    self.personal_best_scores[i] = scores[i]
                    self.personal_best_positions[i] = self.positions[i]

            self.update_archive(scores, self.positions)

            global_best_positions = np.array([sol[0] for sol in self.global_best_archive])
            global_best = global_best_positions[np.random.choice(len(global_best_positions))]

            r1 = np.random.random((self.num_particles, self.n_var))
            r2 = np.random.random((self.num_particles, self.n_var))
            self.velocities = (
                self.w * self.velocities +
                self.c1 * r1 * (self.personal_best_positions - self.positions) +
                self.c2 * r2 * (global_best - self.positions)
            )
            self.positions += self.velocities

            self.positions = np.clip(self.positions, self.lower_bound, self.upper_bound)

            print(f"Generation {gen + 1}/{self.max_gen}: Archive size = {len(self.global_best_archive)}")

        return self.global_best_archive

# **Rastrigin Problem**

Définition du probléme

In [6]:
class RastriginProblem:
    def __init__(self, n_var=2):
        self.n_var = n_var
        self.n_obj = 1
        self.xl = np.array([-5.12] * n_var)
        self.xu = np.array([5.12] * n_var)

    def evaluate(self, x, **kwargs):
        n = x.shape[1]
        f = 10 * n + np.sum(x**2 - 10 * np.cos(2 * np.pi * x), axis=1)
        return {'F': np.array([f]).T}

    def bounds(self):
        return self.xl, self.xu

    def has_constraints(self):
        return False

    def has_bounds(self):
        return True

In [7]:
Rastrigin = RastriginProblem(n_var=2)

Optimisation

In [ ]:
result = minimize(Rastrigin,
                  Algorithm,
                  termination=('n_gen', 15),  # Arreter aprés 100 générations
                  seed=1,
                  verbose=True,
                  save_history = True)

In [ ]:
from IPython.display import display, clear_output
import time

fig = plt.figure(figsize=(10, 10))

history = result.history

for frame in range(len(history)):
    clear_output(wait=True)

    # Create the surface plot
    ax = fig.add_subplot(111, projection='3d')
    x = np.linspace(-5.12, 5.12, 100)
    y = np.linspace(-5.12, 5.12, 100)
    X, Y = np.meshgrid(x, y)
    Z = 10 * 2 + X**2 - 10 * np.cos(2 * np.pi * X) + Y**2 - 10 * np.cos(2 * np.pi * Y)
    surf = ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.5)

    # Plot population at current generation
    pop = history[frame].pop
    solutions = pop.get("X")
    ax.scatter(solutions[:, 0], solutions[:, 1],
              [Rastrigin.evaluate(np.array([x]))['F'][0] for x in solutions],
              color='red', alpha=0.5, s=100)

    ax.set_title(f"Generation {frame}")
    ax.set_xlabel("X1")
    ax.set_ylabel("X2")
    ax.set_zlabel("f(X1, X2)")

    display(plt.gcf())
    time.sleep(0.2)
    plt.clf()

plt.close()

In [ ]:
Best_solution = result.X
print("Best Solution : {}".format(Best_solution))
print("Objective Value : {}".format(result.F[0]))

Visualisation

In [ ]:
x = np.linspace(-5.12, 5.12, 500)
y = np.linspace(-5.12, 5.12, 500)
X, Y = np.meshgrid(x, y)
Z = 10 * 2 + X**2 - 10 * np.cos(2 * np.pi * X) + Y**2 - 10 * np.cos(2 * np.pi * Y)

plt.contourf(X, Y, Z, levels=50, cmap='viridis')
plt.colorbar()
plt.scatter(result.X[0], result.X[1], color='red', label='Best Solution')
plt.title("Rastrigin Function Optimization")
plt.xlabel("X1")
plt.ylabel("X2")
plt.legend()
plt.show()

# **ZDT1**

Définition du probléme

In [12]:
ZDT1 = get_problem("zdt1", n_var=20)

Optimisation

In [ ]:
pso = MultiObjectivePSO(ZDT1)
pareto_archive = pso.optimize()

Visualisation

In [ ]:
pareto_objectives = np.array([obj for _, obj in pareto_archive])

plt.scatter(pareto_objectives[:, 0], pareto_objectives[:, 1], color="red", marker="x")
plt.xlabel("F: 1")
plt.ylabel("F: 2")
plt.title("Pareto Front")
plt.show()